In [1]:
import os
import glob
import pickle

import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [6]:
test_img_path = '/Volumes/CLab/dendritic_scaling/rolling_shutter_test/Corti-hSyn-5f-DIV24-C2-482-TTXPTX-FOV7p9-200fps00069.tif'
test_img_path = '/Users/ykhao/Downloads/rolling_shutter_test/Corti-hSyn-5f-DIV24-C2-482-TTXPTX-FOV7p9-200fps00069.tif'
os.path.exists(test_img_path)

True

In [8]:
img = io.imread(test_img_path).astype('float32')
# permute image in (x,y,t) order
img = np.transpose(img, (1,2,0))
img.shape

(256, 512, 13000)

In [9]:
# Correct the rolling_shutter effect for images
# Assume the rolling shutter effect is in the x direction
# For each row of the image, assign the row with a time
mid_row = (img.shape[0]-1) / 2
mid_row_time = np.arange(1, img.shape[2])  # if the first row starts at time 0, then the mid row starts at time 1
corrected_img = np.zeros_like(img[:,:,:-1])
for row_idx in range(img.shape[0]):
    row_time = np.floor(np.abs(row_idx - mid_row)) / np.floor(mid_row) + np.arange(0, img.shape[2])
    row = img[row_idx]
    corrected_img[row_idx] = interp1d(row_time, row, axis=1, kind='linear')(mid_row_time)

In [12]:
corrected_img = np.transpose(corrected_img, (2,0,1))

In [13]:
# save the corrected image
io.imsave('/Users/ykhao/Downloads/rolling_shutter_test/corrected.tif', corrected_img.astype('uint16'))

/var/folders/2j/4hbq8yj13p1ftfnlwv03r8700000gn/T/ipykernel_13480/1663893780.py:2: UserWarning: /Users/ykhao/Downloads/rolling_shutter_test/corrected.tif is a low contrast image
  io.imsave('/Users/ykhao/Downloads/rolling_shutter_test/corrected.tif', corrected_img.astype('uint16'))


In [2]:
def rolling_shutter_correct(img):
    # input image in (x,y,t) order
    mid_row = (img.shape[0]-1) / 2
    mid_row_time = np.arange(1, img.shape[2])  # if the first row starts at time 0, then the mid row starts at time 1
    corrected_img = np.zeros_like(img[:,:,:-1])
    for row_idx in range(img.shape[0]):
        row_time = np.floor(np.abs(row_idx - mid_row)) / np.floor(mid_row) + np.arange(0, img.shape[2])
        row = img[row_idx]
        corrected_img[row_idx] = interp1d(row_time, row, axis=1, kind='linear')(mid_row_time)
    return corrected_img

In [5]:
# image_dir_list = [
#     '/Volumes/CLab/dendritic_scaling/20231215_B33_neurons_DIV24_dendritic_scaling_2/B2-470/B33-DIV24-B2-470-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231215_B33_neurons_DIV24_dendritic_scaling_2/B3-470/B33-DIV24-B3-470-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231215_B33_neurons_DIV24_dendritic_scaling_2/C2-482/B33-DIV24-C2-482-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231215_B33_neurons_DIV24_dendritic_scaling_2/C3-482/B33-DIV24-C3-482-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231212_B33-neurons-DIV21_dendritic-scaling/Well2/B33-DIV21-corti-glassbottom-well2-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231212_B33-neurons-DIV21_dendritic-scaling/Well3/B33-DIV21-corti-glassbottom-well3-TIFF/TTX',
#     '/Volumes/CLab/dendritic_scaling/20231212_B33-neurons-DIV21_dendritic-scaling/Well4/B33-DIV21-corti-glassbottom-well4-TIFF/TTX']

# image_dir_list = [
#     '/Volumes/MyPassport/dendritic_scaling/20240209/C1_wellB2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240209/C1_wellC2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240209/C1_wellC3/TTX_PTX/']

# image_dir_list = [
#     '/Volumes/MyPassport/dendritic_scaling/20240210/plateC2_WellB2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240210/plateC2_WellB3/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240210/plateC2_WellC2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240210/plateC2_WellC3/TTX_PTX/']
# image_dir_list = [
#     '/Volumes/MyPassport/dendritic_scaling/20240215/plateC3_WellB2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240215/plateC3_WellB3/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240215/plateC3_WellB4/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240215/plateC3_WellC2/TTX_PTX/',
#     '/Volumes/MyPassport/dendritic_scaling/20240215/plateC3_WellC3/TTX_PTX/']
# image_dir_list = [
#     '/Volumes/MyPassport/dendritic_scaling/20240216/plateC2_WellB2/',
#     '/Volumes/MyPassport/dendritic_scaling/20240216/plateC2_WellB3/',
#     '/Volumes/MyPassport/dendritic_scaling/20240216/plateC2_WellC2/',
#     '/Volumes/MyPassport/dendritic_scaling/20240216/plateC2_WellC3/']

image_dir_list = [
    '/Volumes/CLab/dendritic_scaling/20240331/B38_plateC1_wellB2/TTXonly/',
    '/Volumes/CLab/dendritic_scaling/20240331/B38_plateC1_wellC2/TTXonly/']

In [4]:
# for image_dir in image_dir_list:
#     # move all the tif files to a new directory in image_dir called 'raw'
#     raw_dir = os.path.join(image_dir, 'raw')
#     os.makedirs(raw_dir, exist_ok=True)
#     tif_files = glob.glob(os.path.join(image_dir, '*.tif'))
#     for tif_file in tif_files:
#         os.rename(tif_file, os.path.join(raw_dir, os.path.basename(tif_file)))

In [ ]:
for image_dir in image_dir_list:
    print(image_dir)
    image_list = glob.glob(os.path.join(image_dir, 'raw', '*.tif'))
    for image_path in image_list:
        img = io.imread(image_path).astype('uint16')
        img = np.transpose(img, (1,2,0))
        corrected_img = rolling_shutter_correct(img)
        corrected_img = np.transpose(corrected_img, (2,0,1))
        save_path = os.path.join(image_dir, os.path.basename(image_path))
        io.imsave(save_path, corrected_img.astype('uint16'))